In [1]:
%jsroot on

In [2]:
const int lowrun = 563;
const int highrun = 603;
const int _strip = 16;
const int _bin = 50;
const double _minfit = 150;
const double _maxfit = 250;

In [3]:
Double_t fitGaussian(TH1F *h[809][16],Int_t runnum,Int_t ii) {
    // 定义高斯拟合函数
    TF1* gaussian = new TF1("gaussian", "gaus", _minfit, _maxfit);
    h[runnum][ii]->Fit(gaussian, "RQ");

    // 获取拟合结果中的均值
    return gaussian->GetParameter(1);
}

In [4]:
TCanvas *c1=new TCanvas("c1","c1");

In [5]:
        int start = lowrun;
        int stop = highrun;
        TChain *fch;
        fch = new TChain("tree", "tele");
        for (int i= start; i<=stop; i++){
          TString fileName = TString::Format("/data/d1/zhuhy/lanzhou2022/getdatappac/newtest13/tele%04d.root" , i);
          fch->AddFile(fileName.Data());
        }

    // 定义变量用于读取数据
    Int_t runnumber;
    UShort_t taf_ad2_xs[5];
    Double_t taf_ad2_t[5];
    Double_t xT2[2];

    // 设置Branch地址
    fch->SetBranchAddress("runnumber", &runnumber);
    fch->SetBranchAddress("taf_ad5_xs[5]", taf_ad2_xs);
    fch->SetBranchAddress("taf_ad5_t[5]", taf_ad2_t);
    fch->SetBranchAddress("xT2[2]", xT2);

In [6]:
    TH1F *h[809][16];
    Double_t mean[809][16];
    Int_t ccount[809];
    TString find;
    for(Int_t runnum=0;runnum<=809;runnum++){
            ccount[runnum]=0;
        for(Int_t ii=0;ii<16;ii++){
            find = TString::Format("h[%d][%d]",runnum,ii);
            h[runnum][ii] = new TH1F(find.Data(), find.Data(), _bin, _minfit, _maxfit);
            mean[runnum][ii]=0;
        }
    }

In [7]:
 Double_t tof;
 UShort_t st;
// 循环遍历每个事件
  TGraph *g=new TGraph();
  for (int event = 0; event < fch->GetEntries(); event++) {
            // 从TChain中获取当前事件的数据
            st=0;
            if(event%500000==0)cout<<event<<" / "<<fch->GetEntries()<<'\n';
            fch->GetEntry(event);
            // 计算时间差
            tof = taf_ad2_t[0] - xT2[0];
            st = taf_ad2_xs[0];
            // 选择特定的能量范围
            if(runnumber>=0&&runnumber<=809&&st>=0&&st<=15){
                ccount[runnumber]++;
                h[runnumber][st]->Fill(tof);
            }
  }
cout<<"out now!"<<'\n';
    for(Int_t runnum=lowrun;runnum<=highrun;runnum++){
        for(Int_t ii=0;ii<16;ii++){
             mean[runnum][ii]= fitGaussian(h,runnum,ii);   
             g->SetPoint(runnum*16+ii,runnum*16+ii,mean[runnum][ii]);
        }
    }

0 / 6403760
500000 / 6403760
1000000 / 6403760
1500000 / 6403760
2000000 / 6403760
2500000 / 6403760
3000000 / 6403760
3500000 / 6403760
4000000 / 6403760
4500000 / 6403760
5000000 / 6403760
5500000 / 6403760
6000000 / 6403760
out now!


Warning in <Fit>: Fit data is empty 
Warning in <Fit>: Fit data is empty 
Warning in <Fit>: Fit data is empty 
Warning in <Fit>: Fit data is empty 
Warning in <Fit>: Fit data is empty 
Warning in <Fit>: Fit data is empty 
Warning in <Fit>: Fit data is empty 
Warning in <Fit>: Fit data is empty 
Warning in <Fit>: Fit data is empty 
Warning in <Fit>: Fit data is empty 
Warning in <Fit>: Fit data is empty 
Warning in <Fit>: Fit data is empty 
Warning in <Fit>: Fit data is empty 
Warning in <Fit>: Fit data is empty 
Warning in <Fit>: Fit data is empty 
Warning in <Fit>: Fit data is empty 
Warning in <Fit>: Fit data is empty 
Warning in <Fit>: Fit data is empty 
Warning in <Fit>: Fit data is empty 
Warning in <Fit>: Fit data is empty 
Warning in <Fit>: Fit data is empty 
Warning in <Fit>: Fit data is empty 
Warning in <Fit>: Fit data is empty 
Warning in <Fit>: Fit data is empty 
Warning in <Fit>: Fit data is empty 
Warning in <Fit>: Fit data is empty 
Warning in <Fit>: Fit data is empty 
W

In [8]:
/*
    for(Int_t runnum=lowrun;runnum<=highrun;runnum++){
             cout<<runnum<<'\t'<<runnum<<'\n';
        for(Int_t ii=0;ii<16;ii++){
             cout<<ii<<'\t'<<mean[runnum][ii]<<'\n';  
        }
    }
    */

In [9]:
        ofstream write1;
        TString fileName = TString::Format("/home/zhuhy/JNBook/Experiment2022/taf/tofdE/txt/ad5_offset_563_603.txt");
        write1.open(fileName.Data(), ios::out);
        for(Int_t runnum=lowrun;runnum<=highrun;runnum++){
            write1<<runnum<<'\t'<<ccount[runnum]<<endl;
            for(Int_t ii=0;ii<16;ii++){
                 write1<<ii<<'\t'<<mean[runnum][ii]<<endl;  
            }
        }
        write1.close();
